In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import time

In [2]:
df1 = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5'),
    ('id_cliente-3',  'cat-6, cat-7'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9'),
    ('id_cliente-9',  'cat-1'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

In [3]:
df2 = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3, cat-15'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5, cat-11, cat-14'),
    ('id_cliente-3',  'cat-4, cat-14, cat-15'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10, cat-11'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10, cat-11, cat-13'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9, cat-12, cat-13, cat-14'),
    ('id_cliente-9',  'cat-2'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

In [4]:
def desafio(df):
  df.cache()
  produtos = []
  for row in df.select(split(df['categorias'], pattern=',')).alias('produtos_split').collect():
    produtos = produtos + row[0]
  produtos = list(set(produtos))
  produtos2 = [int(produto.strip().replace(prefix + '-','')) for produto in produtos]
  produtos2.sort()
  prefixo = produtos[0].split('-')[0]
  base =  'client-uuid-'
  df_final = df
  df_final = df_final.withColumn('id',  concat(lit(base),split(df['id_cliente'], '-')[1]))
  for produto in produtos2:
    column_name = prefixo + '-' + str(produto)
    df_final = df_final.withColumn(column_name, when(col("categorias").contains(column_name), lit(1)).otherwise(lit(0)))
  df_final.drop('categorias').drop('id_cliente').show()

In [5]:
start = time.time()
desafio(df1)
desafio(df2)
print(time.time() - start)

+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
 id|cat-1|cat-2|cat-3|cat-4|cat-5|cat-6|cat-7|cat-8|cat-9|cat-10|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
 client-uuid-1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0|
 client-uuid-2| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0|
 client-uuid-3| 0| 0| 0| 0| 0| 1| 1| 0| 0| 0|
 client-uuid-4| 1| 1| 0| 0| 0| 0| 1| 0| 0| 1|
 client-uuid-5| 1| 0| 0| 0| 0| 0| 0| 1| 0| 1|
 client-uuid-6| 1| 0| 0| 0| 0| 0| 0| 0| 1| 1|
 client-uuid-7| 1| 0| 0| 1| 1| 0| 0| 0| 0| 1|
 client-uuid-8| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0|
 client-uuid-9| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
client-uuid-10| 1| 1| 1| 1| 1| 1| 1| 1| 0| 1|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+

+--------------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+
 id| cat-1| cat-2| cat-3| cat-4| cat-5| cat-6| cat-7| cat-8| cat-9| cat-10| cat-11| cat-12| cat-13| cat-14| cat-15|
+--------------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+
 client-uuid-1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1|
 client-uuid-2| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0|
 client-uuid-3| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 1|
 client-uuid-4| 1| 1| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0|
 client-uuid-5| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 1| 0| 0| 0| 0|
 client-uuid-6| 1| 0| 0| 0| 0| 0| 0| 0| 1| 1| 1| 0| 1| 0| 0|
 client-uuid-7| 1| 0| 0| 1| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0|
 client-uuid-8| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 1| 1| 1| 0|
 client-uuid-9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
client-uuid-10| 1| 1| 1| 1| 1| 1| 1| 1| 0| 1| 0| 0| 0| 0| 0|
+--------------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+

0.9807634353637695